*In this notebook, we will start to learn how to deal with polynomials in Oscar, and in particular, how do carry out polynomial division.*

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.13.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


## Polynomials in Oscar

The first step for doing polynomial algebra in Oscar is always to specify the **polynomial ring** we will work in.

<div class="alert alert-block alert-info">
    <strong>Note:</strong> Contrary to what we said in a previous version of this notebook, we recommend that you <strong>don't</strong> specify any monomial ordering when you define a polynomial ring. 
    This is because many Oscar functions (e.g., <code>leading_term</code>) simply ignore orderings defined on rings, and instead require that you give the ordering as an input.
</div>

### Example 1: Indexed variables

If we want the polynomial ring $S=\mathbb{Z}[z_1,\ldots,z_5]$, and a polynomial $g= 3z_{1}z_{2}^2+5z_{3}z_{4}z_{5}^2\in S$, we write the following:

In [2]:
S, z = polynomial_ring(ZZ, "z"=>1:5)

(Multivariate polynomial ring in 5 variables over ZZ, ZZMPolyRingElem[z_{1}, z_{2}, z_{3}, z_{4}, z_{5}])

In [3]:
g = 3*z[1]*z[2]^2 + 5*z[3]*z[4]*z[5]^2

3*z_{1}*z_{2}^2 + 5*z_{3}*z_{4}*z_{5}^2

In [4]:
g(1,2,1,1,1)

17

### Example 2: Named variables

In the rest of the notebook, we will work in the polynomial ring $R=\mathbb{Q}[x,y]$, and the polynomial $f=x^2 + xy^2 + y$.

In [5]:
R, (x,y) = polynomial_ring(QQ, ["x","y"])

(Multivariate polynomial ring in 2 variables over QQ, QQMPolyRingElem[x, y])

In [6]:
f = x^2 + x*y^2 + y

x^2 + x*y^2 + y

In [7]:
f(1,2)

7

## Leading terms

We obtain the **leading term**, **leading monomial** and **leading coefficient** of a polynomial $f\in k[x_1,\ldots,x_n]$ with the following commands.

Note that we need to specify with respect to what **monomial order** on our ring we want to do this.

Some examples of monomial orderings that we can put on $R=k[x_1,\ldots,x_n]$ are the following:

- `lex(R)` (lexicographic ordering, $<_\text{lex}$)
- `deglex(R)`(graded lexicographic ordering, $<_\text{grlex}$)
- `degrevlex(R)`(graded lexicographic ordering, $<_\text{grevlex}$)
- `matrix_ordering(R,M)` (the matrix ordering $<_M$ from Exercise 8, for some real-valued matrix $M$ with $n$ columns)

<div class="alert alert-block alert-info">
<strong>Cool fact:</strong> Robbiano proved in his paper <em>Term orderings on the polynomial ring</em> (EUROCAL, 1985) that any monomial order can be realized as $<_M$ for some appropriately chosen matrix $M$.
</div>
  

In [8]:
f

x^2 + x*y^2 + y

In [9]:
leading_term(f, ordering=lex(R))

x^2

In [10]:
leading_term(f, ordering=deglex(R))

x*y^2

In [11]:
leading_term(f, ordering=degrevlex(R))

x*y^2

In [12]:
M = [1 1;2 1];
leading_term(f, ordering=matrix_ordering(R,M))

x*y^2

## The multivariate division algorithm

<div class="alert alert-block alert-info">
<strong>Warning:</strong> In a previous version of this notebook we recommended the Oscar function <code>divrem</code> for multivariate divison, which has the unusual property that the user cannot supply a monomial order as input. Instead, the monomial order needs to be defined <strong>already when the ring is defined</strong>. Since this is inconvenient and easilly causes confusion, we recommend that you don't use <code>divrem</code>.
</div>

Instead of using `divrem`, we here write our own implementation of the pseudocode for the multivariate polynomial division given on the exercise sheet. 

Feel free to copy and paste this into other notebooks if you end up needing multivariate polynomial division for something else.

*Note:* This code is quite a bit more complicated than what we expect you to be able to produce yourself, so don't worry if you don't understand all the details. But try at least to compare the overall structure to the pseudocode from the exercise sheet.

In [13]:
function multivariate_division(f::MPolyRingElem, F::Vector{<:MPolyRingElem}; ordering::MonomialOrdering)
    LT = (h -> leading_term(h,ordering=ordering))
    s = length(F)
    Q = [zero(parent(f)) for i=1:s]
    r = 0
    p = f
    while p != 0
        division_occured = false
        for i = 1:s
            division_occured, quotient = divides( LT(p), LT(F[i]) )
            if division_occured 
                Q[i] = Q[i] + quotient
                p = p - quotient*F[i]
                break
            end
        end
        if !(division_occured)
            r = r + LT(p)
            p = p - LT(p)
        end
    end
    return Q,r
end;

## Example of multivariate division

Let's keep our polynomial $f$ from before, and suppose that we want to divide it by the following tuple $F=(f_1,f_2)$:

In [14]:
f

x^2 + x*y^2 + y

In [15]:
f1 = x + x*y
f2 = x+y
F = [f1,f2]

2-element Vector{QQMPolyRingElem}:
 x*y + x
 x + y

In [16]:
Q,r = multivariate_division(f, [f1,f2], ordering = lex(R))

(QQMPolyRingElem[y - 2, x + 2], -y)

The quotient is this following vector:

In [17]:
Q

2-element Vector{QQMPolyRingElem}:
 y - 2
 x + 2

The remainder is the following polynomial:

In [18]:
r

-y

Let's double-check that the ouput is correct, in the sense that $f=Q\bullet F+r$.

In [19]:
dot(Q,F) + r - f

0

## (Coefficients and monomials)

This is not needed for the exercise session in Week 1, but it's still good to know about.

The functions `coefficients()` and `monomials()` extract the **coefficients** and the **monomials** of a polynomial. 

**Note:** The output of these functions are *iterators*. To turn them into list, one can use the function `collect()`. You can also acess the values in an iterator through a for loop.


<div class="alert alert-block alert-info">
<strong>Also note:</strong> Julia automatically prints the output of the last command run in a cell (unless this is supressed with <code>;</code>). 
If you want to print other outputs, you can use the commands <code>display</code> or <code>println</code> ("ln" stands for <em>line</em>, and means that the output is printed on a separate line).
</div>

In [20]:
monomials(f)

monomials iterator of x*y^2 + x^2 + y

In [21]:
M = collect(monomials(f))

3-element Vector{QQMPolyRingElem}:
 x*y^2
 x^2
 y

In [22]:
M[1]

x*y^2

In [23]:
M[2]

x^2

In [24]:
M[3]

y

In [25]:
for a in monomials(f)
    println(a)
end    

x*y^2
x^2
y


In [26]:
C = collect(coefficients(f))

3-element Vector{QQFieldElem}:
 1
 1
 1

We can reconstruct our polynomial from the coefficients and monomials as follows:

In [27]:
sum([C[i]*M[i] for i=1:length(C)])

x^2 + x*y^2 + y

We can also use a dot product, like this:

In [28]:
dot(C,M)

x^2 + x*y^2 + y